In [2]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
import pathlib
pathlib.Path().resolve()

WindowsPath('D:/Materi Kuliah/Semester 7/Sistem Temu Kembali Informasi/Proposal/app')

In [7]:
import pandas as pd

ms = pd.read_csv('musicFinder_2.csv')
ms.tail(10)

,artist_name,track_name,release_date,genre,lyrics,len,dating,violence,world/life,night/time,...,sadness,feelings,danceability,loudness,acousticness,instrumentalness,valence,energy,topic,age
28362,zz top,la grange,1973,blues,rumor spreadin round texas bout shack outside ...,26,0.002770,0.002770,0.002770,0.153889,...,0.002770,0.002770,0.523448,0.659436,0.002499,0.030769,0.600165,0.640629,obscene,0.671429
28363,zz top,waitin' for the bus,1973,blues,wait wait paper takehome pack tight pack tight...,15,0.003759,0.003759,0.003759,0.482210,...,0.003759,0.003759,0.692408,0.650053,0.046485,0.084514,0.861913,0.371352,night/time,0.671429
28364,zz top,beer drinkers & hell raisers,1973,blues,life take life care betray true friend betray ...,92,0.000658,0.076494,0.222353,0.000658,...,0.361336,0.062980,0.468212,0.696177,0.001405,0.005263,0.582646,0.756749,sadness,0.671429
28365,zz top,brown sugar,1970,blues,friends tell go change life friends tell go ch...,52,0.000993,0.000993,0.403845,0.000993,...,0.000993,0.358509,0.568938,0.624055,0.663654,0.086336,0.540396,0.413395,world/life,0.714286
28366,zz top,squank,1970,blues,woman grab children hide catch fight stay aliv...,43,0.001880,0.324093,0.205639,0.001880,...,0.001880,0.001880,0.849453,0.589032,0.573293,0.158907,0.967024,0.304283,violence,0.714286
28367,zz top,old man,1970,blues,sleep away dream yesterday come care head tomo...,32,0.001880,0.139154,0.345720,0.001880,...,0.001880,0.001880,0.279757,0.615235,0.133533,0.344130,0.097279,0.342322,world/life,0.714286
28368,дэфолиант,валуев,2016,hip hop,feel wall fall feel lose feel sweetest sound f...,81,0.000810,0.239508,0.000810,0.020497,...,0.165541,0.348297,0.798549,0.787375,0.133533,0.000004,0.714551,0.939938,feelings,0.057143
28369,張露,夢斷巫山,1958,pop,remember remember thumb yeah baby time come ba...,45,0.162737,0.133445,0.315612,0.001422,...,0.001422,0.001422,0.411892,0.638259,0.909638,0.000791,0.593982,0.383364,world/life,0.885714
28370,張露,你去了那裡,1958,pop,right outside lazy summer home time soul criti...,95,0.000786,0.000786,0.113374,0.062054,...,0.215052,0.000786,0.559190,0.657180,0.937751,0.000007,0.846455,0.459443,music,0.885714
28371,彭家麗,從不喜歡孤單一個,1991,pop,lonely sidewalks silent night bring even deep ...,56,0.136184,0.001053,0.001053,0.301280,...,0.200115,0.040954,0.662082,0.698433,0.663654,0.000000,0.303380,0.503488,night/time,0.414286


In [9]:
ms = ms[ms['release_date'] >= 2000]
ms = ms[['artist_name', 'track_name','release_date', 'genre', 'lyrics', 'topic']]
ms.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10412 entries, 0 to 28368
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   artist_name   10412 non-null  object
 1   track_name    10412 non-null  object
 2   release_date  10412 non-null  int64 
 3   genre         10412 non-null  object
 4   lyrics        10412 non-null  object
 5   topic         10412 non-null  object
dtypes: int64(1), object(5)
memory usage: 569.4+ KB


In [10]:
import re
import numpy as np
import contractions

stop_words = nltk.corpus.stopwords.words('english')

def normalize_document(doc):
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z0-9\s]', '', doc, re.I|re.A)
    doc = doc.lower()
    doc = doc.strip()
    doc = contractions.fix(doc)
    # tokenize document
    tokens = nltk.word_tokenize(doc)
    #filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    return doc

normalize_corpus = np.vectorize(normalize_document)

norm_corpus = normalize_corpus(list(ms['lyrics']))
len(norm_corpus)

10412

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer(ngram_range=(1, 2), min_df=2)
tfidf_matrix = tf.fit_transform(norm_corpus)
tfidf_matrix.shape

(10412, 79832)

In [12]:
from sklearn.metrics.pairwise import cosine_similarity

doc_sim = cosine_similarity(tfidf_matrix)
doc_sim_df = pd.DataFrame(doc_sim)
doc_sim_df.head()

,0,1,2,3,4,5,6,7,8,9,...,10402,10403,10404,10405,10406,10407,10408,10409,10410,10411
0,1.000000,0.013118,0.008033,0.023461,0.003100,0.021755,0.002572,0.006374,0.009909,0.004182,...,0.013466,0.000566,0.070741,0.001621,0.059842,0.015162,0.001185,0.000000,0.000776,0.008350
1,0.013118,1.000000,0.064869,0.032799,0.018096,0.007445,0.005797,0.027095,0.003170,0.006233,...,0.022474,0.023476,0.051174,0.015596,0.024558,0.040724,0.000000,0.008391,0.008177,0.024750
2,0.008033,0.064869,1.000000,0.022204,0.005558,0.002997,0.003078,0.014792,0.001244,0.005147,...,0.014561,0.001187,0.007274,0.007714,0.001810,0.013942,0.005279,0.013457,0.002314,0.006252
3,0.023461,0.032799,0.022204,1.000000,0.004676,0.032990,0.012561,0.007859,0.004845,0.004221,...,0.052915,0.003557,0.013805,0.012357,0.037879,0.039299,0.012030,0.000000,0.011475,0.022121
4,0.003100,0.018096,0.005558,0.004676,1.000000,0.000715,0.004175,0.018451,0.000516,0.000335,...,0.012979,0.001409,0.001899,0.001331,0.006181,0.011532,0.004470,0.000000,0.000000,0.005782


In [13]:
ms.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10412 entries, 0 to 28368
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   artist_name   10412 non-null  object
 1   track_name    10412 non-null  object
 2   release_date  10412 non-null  int64 
 3   genre         10412 non-null  object
 4   lyrics        10412 non-null  object
 5   topic         10412 non-null  object
dtypes: int64(1), object(5)
memory usage: 569.4+ KB


In [14]:
song_list = ms['track_name'].values
song_list, song_list.shape

(array(['when your heart stops beating', 'appel local', 'leçon 3', ...,
        'cossa nostra', 'i gotsta get paid', 'валуев'], dtype=object),
 (10412,))

In [17]:
inputSong = input('Masukan lagu : ')
print("Lagu yg dicari : ", inputSong)
song_id = np.where(song_list == inputSong)[0][0]
song_id

Lagu yg dicari :  sunflower


6903

In [18]:
song_similarities = doc_sim_df.iloc[song_id].values
song_similarities

array([0.01124287, 0.03417918, 0.01260208, ..., 0.00429707, 0.01023859,
       0.01310116])

In [19]:
similar_song_id = np.argsort(-song_similarities)[1:6]
similar_song_id

array([5908, 2882, 2508, 5163, 2493], dtype=int64)

In [20]:
similar_song = song_list[similar_song_id]
similar_song

array(['wrecking ball', 'dust my broom', 'real hip hop', 'either way',
       'hellohello'], dtype=object)

In [37]:
def music_finder(song_name, songs = song_list, doc_sim=doc_sim_df):
    song_id = np.where(songs == song_name)[0][0]
    song_similarities = doc_sim.iloc[song_id].values
    similar_song_id = np.argsort(-song_similarities)[1:6]
    similar_song = songs[similar_song_id]
    return similar_song

In [43]:
song = input("Judul lagu: ")
result = music_finder(song_name = song, songs = song_list, doc_sim=doc_sim_df)
print("Top 5 rekomen lagu: ", result ,"\n")
detail = ms.loc[ms['track_name'].isin(result)]
sorted_detail = detail.sort_values(by='track_name')
for i in result:
    print(i)
    if i in ms['track_name'].values:
        detail = ms[ms['track_name'] == i]
        print(detail, "\n")

Top 5 rekomen lagu:  ['the pusher' 'whole damn town' "you're such a" 'god damn liar'
 'out, damn light'] 

the pusher
         sid        artist_name  track_name  release_date  genre  \
14190  14191  left lane cruiser  the pusher          2016  blues   

                                                  lyrics    topic  
14190  know smoke grass pop couple pills nothin spiri...  obscene   

whole damn town
         sid           artist_name       track_name  release_date    genre  \
26716  26717  turnpike troubadours  whole damn town          2010  country   

                                                  lyrics    topic  
26716  downtown drink sing country songs daylight fin...  obscene   

you're such a
         sid       artist_name     track_name  release_date genre  \
10052  10053  hailee steinfeld  you're such a          2016   pop   

                                                  lyrics    topic  
10052  call call say want love love try try sweet tal...  obscene   

god d

In [42]:
song = input("Judul lagu: ")
print("Lagu yang dicari : ", song)
result = music_finder(song_name = song, songs = song_list, doc_sim=doc_sim_df)
print("Top 5 rekomen lagu: ", result ,"\n")
detail = ms.loc[ms['track_name'].isin(result)]
#sorted_detail = detail.sort_values(by='track_name')

for index, row in detail.iterrows():
    artist_name = row['artist_name']
    track_name = row['track_name']
    release_date = row['release_date']
    genre = row['genre']
    topic = row['topic']

    # Print song details
    print("{:<20} {:<20} {:<10} {:<10} {:<10}".format(artist_name, track_name, release_date, genre, topic))

Lagu yang dicari :  sunflower
Top 5 rekomen lagu:  ['wrecking ball' 'dust my broom' 'real hip hop' 'either way' 'hellohello'] 

beta radio           either way           2010       pop        night/time
dj grumble           hellohello           2018       jazz       violence  
dj tomekk            real hip hop         2013       hip hop    violence  
etta james           dust my broom        2004       blues      violence  
joe walsh            wrecking ball        2012       blues      world/life
lee ann womack       either way           2008       country    sadness   
lil' flip            real hip hop         2018       hip hop    obscene   
miley cyrus          wrecking ball        2013       pop        sadness   
mother mother        wrecking ball        2008       rock       violence  
